In [ ]:
#import 
import pandas as pd
import json
import datetime as dt
from nsepy import get_history as stock_hist
import numpy as np
from pycaret.regression import *

In [ ]:
#Utils functions
def read_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)
    
def date_parser(dt):
    dt = dt.split(',')
    year = int(dt[0])
    month = int(dt[1])
    day = int(dt[2])
    return year, month, day

In [ ]:
config_file = read_json('config.json')

In [ ]:
symbol = config_file['symbol']

y, m, d = date_parser(config_file['train_start_date'])
train_start_date = dt.datetime(y,m,d)

y, m, d = date_parser(config_file['train_end_date'])
train_end_date = dt.datetime(y,m,d)

y, m, d = date_parser(config_file['test_start_date'])
test_start_date = dt.datetime(y,m,d)

y, m, d = date_parser(config_file['test_end_date'])
test_end_date = dt.datetime(y,m,d)

In [ ]:
#Setting start and end dates and fetching the historical data

stk_data = stock_hist(symbol=symbol,start=train_start_date,end=test_end_date) #considering both train and test data

In [ ]:
stk_data.head()

In [ ]:
stk_data['Date'] = stk_data.index
data = pd.DataFrame(columns = ['Date', 'Open'])
data['Date'] = stk_data['Date']
data['Open'] = stk_data['Open']

In [ ]:
# create 12 month moving average
data['MA12'] = data['Open'].rolling(12).mean()
# plot the data and MA
import plotly.express as px
fig = px.line(data, x='Date', y=["Open","MA12"], template = 'plotly_dark')
fig.show()

In [ ]:
# extract month and year from dates
data['Date'] = data.index
data['Month'] = [i.month for i in data['Date']]
data['Year'] = [i.year for i in data['Date']]
data['Day'] = [i.day for i in data['Date']]
# create a sequence of numbers
data['Series'] = np.arange(1,len(data)+1)
# drop unnecessary columns and re-arrange
data = data[['Series', 'Year', 'Month','Day','Open']] 
# check the head of the dataset
data.head()

In [ ]:
data.tail()

In [ ]:
# split data into train-test set
train = data[data['Year'] < 2020]
test = data[data['Year'] >= 2020]
# check shape
train.shape, test.shape


In [ ]:
# initialize setup
s = setup(data = train, test_data = test, target = 'Passengers', fold_strategy = 'timeseries', 
          numeric_features = ['Year', 'Series'], fold = 3, transform_target = True, session_id = 123)